## Домашнее задание:

Задача поиск похожих по эмбеддингам


1. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов. На вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

2. Проверить насколько хорошо работают подходы


In [ ]:
import gensim.downloader as api

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install razdel pymorphy2

In [ ]:
! pip install annoy

In [ ]:
import pandas as pd
import string
from pymorphy2 import MorphAnalyzer
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
tqdm.pandas()

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d phoenix120/gazeta-summaries

gazeta-summaries.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/gazeta-summaries.zip

Archive:  /content/gazeta-summaries.zip
replace gazeta_test.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gazeta_test.jsonl       
replace gazeta_train.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gazeta_train.jsonl      y

replace gazeta_val.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: gazeta_val.jsonl        


In [ ]:
stopword_ru = stopwords.words('russian')

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c' -O stopwords.txt

--2023-07-28 13:32:08--  https://drive.google.com/uc?export=download&id=17wVn5YPpMjHToctGgff_KfSeWcIIlf7c
Resolving drive.google.com (drive.google.com)... 108.177.121.102, 108.177.121.100, 108.177.121.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.121.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lkvmht79neesnfu5brcvi4kuar5frb7p/1690551075000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=6c6d6eae-b024-4cbd-acf4-168a7f2731e9 [following]
--2023-07-28 13:32:08--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lkvmht79neesnfu5brcvi4kuar5frb7p/1690551075000/14904333240138417226/*/17wVn5YPpMjHToctGgff_KfSeWcIIlf7c?e=download&uuid=6c6d6eae-b024-4cbd-acf4-168a7f2731e9
Resolving doc-0s-c0-docs.googleusercontent.com (doc-0s-c0-docs.googleusercontent.com)... 209.85.145.132, 2

In [ ]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]

stopword_ru += additional_stopwords

In [ ]:
jsonObj = pd.read_json(path_or_buf='/content/gazeta_train.jsonl', lines=True)

In [ ]:
jsonObj.to_csv('gazeta_train.csv')

In [ ]:
jsonVal = pd.read_json(path_or_buf='/content/gazeta_val.jsonl', lines=True)
jsonVal.to_csv('gazeta_val.csv')

In [ ]:
df = pd.read_csv('gazeta_train.csv')
df.head()

,Unnamed: 0,date,url,summary,title,text
0,0,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...,С 2011 года правительство отменяет самый раздр...,Налог в бак,Сегодня транспортный налог начисляется в завис...
1,1,2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",Словосочетание «музыкальный кинофестиваль» уже...
2,2,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,После более чем 12-часовых консультаций Совет ...
3,3,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...,Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,"Жертвами урагана «Агата», обрушившегося на Цен..."
4,4,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,Решение ограничить рост тарифов естественных м...


In [ ]:
df_val = pd.read_csv('gazeta_val.csv')
df_val.head()

,Unnamed: 0,date,url,summary,title,text
0,0,2020-01-01 10:29:58,https://www.gazeta.ru/business/2019/12/26/1288...,В уходящем году инфляция в России находится на...,Дорогой 2020-й: какие продукты подскочат в цене,"В 2020 году инфляция в России составит 3,5-4%,..."
1,1,2020-01-01 11:16:44,https://www.gazeta.ru/army/2020/01/01/12894224...,Мировая общественность призвала лидера КНДР Ки...,Подарок от Ким Чен Ына: Трамп ответил на новые...,Глава Белого дома Дональд Трамп выразил надежд...
2,2,2020-01-01 12:39:08,https://www.gazeta.ru/politics/2020/01/01_a_12...,Главный раввин Польши Михаил Шудрих раскритико...,Это манипуляция: раввин Польши ответил Путину ...,Председатель Союза еврейских религиозных общин...
3,3,2020-01-01 13:36:41,https://www.gazeta.ru/culture/2020/01/01/a_128...,Народная артистка СССР София Ротару выступила ...,Первая после Путина: Ротару выступила на Новый...,Первой песней наступившего 2020 года на Первом...
4,4,2020-01-01 14:20:16,https://www.gazeta.ru/politics/2019/12/30_a_12...,1 января 1895 года родился самый могущественны...,"Пресли, Леннон, Трумэн: против кого работал Эд...","Выражение «глубинное государство», пришедшее и..."


In [ ]:
import re
from razdel import tokenize
import pymorphy2

In [ ]:
def clean_text(text):
    '''
    очистка текста

    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    text = re.sub('n', ' ', text)

    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист лемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w) > 1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [i for i in words_lem if not i in stopword_ru] # [6]

    return words_lem_without_stopwords

In [ ]:
df['text'] = df['text'].progress_apply(lambda x: clean_text(x))

  0%|          | 0/60964 [00:00<?, ?it/s]<ipython-input-20-9f6154e12c19>:14: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
100%|██████████| 60964/60964 [01:53<00:00, 534.96it/s]


In [ ]:
df['text'] = df['text'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 60964/60964 [11:23<00:00, 89.23it/s] 


In [ ]:
texts = list(df['text'].values)

In [ ]:
df_val['text'] = df_val['text'].progress_apply(lambda x: clean_text(x))

100%|██████████| 6369/6369 [00:13<00:00, 482.19it/s]


In [ ]:
df_val['text'] = df_val['text'].progress_apply(lambda x: lemmatization(x))

100%|██████████| 6369/6369 [01:04<00:00, 99.12it/s] 


In [ ]:
texts_val = list(df_val['text'].values)

In [ ]:
modelW2V = Word2Vec(sentences=texts, vector_size=300, window=5, min_count=1)

In [ ]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
#ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in texts_val:
    n_w2v = 0
    #n_ft = 0
    #spls = line.split("\t")
    index_map[counter] = line[1]
    #question = preprocess_txt(spls[0])

    vector_w2v = np.zeros(300)
    #vector_ft = np.zeros(300)
    for word in line:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        # if word in modelFT.wv:
        #     vector_ft += modelFT.wv[word]
        #     n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    # if n_ft > 0:
    #     vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    #ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
#ft_index.build(10)

True

In [ ]:
def get_response(question, index, model, index_map):
    question = clean_text(question)
    question = lemmatization(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [ ]:
TEXT = "политическая обстановка в США"

In [ ]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['массовый', 'brexit', 'сша']

In [ ]:
#get_response(TEXT, ft_index, modelFT, index_map)